In [1]:
class News:
    def __init__(self, title, image, pubDate, link):
        self.title = title
        self.image = image
        self.pubDate = pubDate
        self.link = link

In [2]:
from urllib.request import urlopen, Request
from xml.etree import ElementTree

In [ ]:
def get_news():
    headers = {'User-Agent': 'Mozilla/5.0'}
    url = 'https://news.yahoo.com/rss/all'
    req = Request(url, headers=headers, method='GET')

    with urlopen(req) as resp:
        data = ElementTree.fromstring(resp.read().decode('utf-8'))
        channel = data[0]

        news_links = []
        for child in channel:
            if child.tag == 'item':
                try:
                    news_links.append(process_news_item(child))
                except Exception as ex:
                    print('Item invalid')

    return news_links

In [ ]:
def render_news_item(item):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        with urlopen(Request(item.link, headers=headers, method='GET')) as resp:
            html_fragment = (f'<a href="{item.link}"><h3>               {item.title}</h3></a>'
                            f'<img src="{item.image}" width="50px" height="50px" />'
                             f'<input type="hidden" name="sec" value="aaa" />')

            return html_fragment
    except Exception as ex:
        return ""

In [ ]:
from time import time
start = time()
articles_html = ""
for item in news:
    articles_html += render_news_item(item)

print(f'Job finished for {len(news)} news sequentially in {time() - start} seconds')

In [ ]:
from threading import Thread
class HtmlFragmenter(Thread):
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
        self._return = None

    def run(self):
        while True:
            news_item = self.queue.get()
            self._return = render_news_item(news_item)
            self.queue.task_done()

    def join(self, *args):
        Thread.join(self, *args)
        return self._return